In [ ]:
from fuzzywuzzy import fuzz, process
import os
import pandas as pd
import pathlib as Path
from simple_cysh import *
#%matplotlib inline

In [ ]:
heat_map_folders = ["Pride Heat Maps_ Coaching Logs",
                    "Spirit Heat Maps_ Coaching Logs",
                    "Purpose Heat Maps_ Coaching Logs",
                    "Heat Maps _ Coaching Logs"]

# for x in os.walk("Z:\\Heat Maps _ Coaching Logs"):
#     print(x)
coach_xlsx_paths = [r'Z:\Pride Heat Maps_ Coaching Logs\Herzl\SY18_BaconCoachingLog_Herzl.xlsx',
                    r'Z:\Pride Heat Maps_ Coaching Logs\Langford\SY18_BaconCoachingLog_Langford.xlsx',
                    r'Z:\Pride Heat Maps_ Coaching Logs\Chicago Funston\Funston. FY18 Coaching Log (1).xlsx',
                    r'Z:\Pride Heat Maps_ Coaching Logs\Chalmers\SY18_BaconCoachingLog_Chalmers.xlsx',
                    r'Z:\Pride Heat Maps_ Coaching Logs\Johnson\SY18_BaconCoachingLog_Johnson.xlsx',
                    r'Z:\Pride Heat Maps_ Coaching Logs\Collins\SY18_BaconCoachingLog_Collins.xlsx',
                    r'Z:\Pride Heat Maps_ Coaching Logs\Dvorak\SY18_BaconCoachingLog_Dvorak.xlsx',
                    r"Z:\Spirit Heat Maps_ Coaching Logs\Chicago O'Keeffe\SY18CoachingLog_OKeeffe.xlsx",
                    r'Z:\Spirit Heat Maps_ Coaching Logs\Chicago Dulles\SY18CoachingLog_Dulles.xlsx',
                    r'Z:\Spirit Heat Maps_ Coaching Logs\Chicago Gregory\SY18CoachingLog_Gregory.xlsx',
                    r'Z:\Spirit Heat Maps_ Coaching Logs\Chicago  Phillips\SY18CoachingLog_Phillips.xlsx',
                    #'Z:\\Spirit Heat Maps_ Coaching Logs\Chicago Spencer\SY18CoachingLog_Spencer CAM.xlsx', ## Which?
                    r'Z:\Spirit Heat Maps_ Coaching Logs\Chicago Spencer\SY18CoachingLog_Spencer.xlsx',
                    #'Z:\\Spirit Heat Maps_ Coaching Logs\\Chicago Stagg\Coaching Log_stagg.xlsx',
                    r'Z:\Spirit Heat Maps_ Coaching Logs\Chicago Stagg\SY18CoachingLog_Stagg.xlsx',
                    #'Z:\\Spirit Heat Maps_ Coaching Logs\Chicago Stagg\stagg heat map.xlsx',
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Gage Park\\SY18CoachingLog_Gage Park.xlsx',
                    r'Z:\Purpose Heat Maps_ Coaching Logs\ Chicago Burke\SY18CoachingLog_Burke.xlsx',
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Jordan\Jordan.FY18 Coaching Log (1).xlsx',
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Deneen\SY18CoachingLog_Deneen.xlsx',
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Sullivan\SY18CoachingLog_Sullivan.xlsx',
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Pershing East\SY18CoachingLog_Pershing_Sem2.xlsx', ## Which?
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Pershing East\SY18CoachingLog_Pershing.xlsx', ## Which?
                    r'Z:\Purpose Heat Maps_ Coaching Logs\Chicago Gale\SY18CoachingLog_Gale.xlsx',
                    r'Z:\Heat Maps _ Coaching Logs\Chicago Piccolo\Piccolo Heat Map Coaching Log.xlsx',
                    r'Z:\Heat Maps _ Coaching Logs\Chicago Academy HS\Chicago Academy Heat Map Coaching Log.xlsx',
                    r'Z:\Heat Maps _ Coaching Logs\Chicago Lovett\Lovett Heat Map Coaching Log.xlsx',
                    r'Z:\Heat Maps _ Coaching Logs\Chicago McNair\McNair Heat Map Coaching Log.xlsx',
                    r'Z:\Heat Maps _ Coaching Logs\Chicago Fuller\Fuller Heat Map Coaching Log.xlsx',
                    r'Z:\Heat Maps _ Coaching Logs\Chicago Woodson\Woodson Heat Map Coaching Log.xlsx']

sheet_months = ['Oct','Nov','Dec','Jan','Feb','March','April','May']

expected_cols = [('Month', ''),
                 ('ACM', ''),
                 ('        Effective Tutoring \xa0 \xa0 \xa0 \xa0 \xa0     \xa0(ELA and Math Strategies)','Rating'),
                 ('        Effective Tutoring \xa0 \xa0 \xa0 \xa0 \xa0     \xa0(ELA and Math Strategies)','Rubric Note'),
                 ('Application of Feedback and Training', 'Rating'),
                 ('Application of Feedback and Training', 'Rubric Note'),
                 ('Effective Student Engagement & Behavior Management', 'Rating'),
                 ('Effective Student Engagement & Behavior Management', 'Rubric Note'),
                 ('Lesson Planning and Preparation', 'Rating'),
                 ('Lesson Planning and Preparation', 'Rubric Note'),
                 ('Student Progress Monitoring', 'Rating'),
                 ('Student Progress Monitoring', 'Rubric Note')]

declare_cols = ['ACM',
                'Plan Rating',
                'Plan Rubric Note',
                'ET Rating',
                'ET Rubric Note',
                'ESE Rating',
                'ESE Rubric Note',
                'SPM Rating',
                'SPM Rubric Note',
                'Learn Rating',
                'Learn Rubric Note',
                'Month',
                'Folder']

In [ ]:
# Gather heatmap data
# want to flag which docs have headers out of order, or named different than expected
d = []
for filepath in coach_xlsx_paths:
    # use pd.ExcelFile instead of pd.read_excel in order to use variable sheetnames
    with pd.ExcelFile(filepath) as xl:
        for sheet in list(set(xl.sheet_names) & set(sheet_months)):
            # Returns ordered dict
            heatmap = pd.read_excel(xl, header=[2,3], 
                                    sheet_name=sheet, 
                                    usecols=14, index_col=None)
            heatmap = heatmap.reset_index()
            heatmap = heatmap.rename(columns={heatmap.columns.get_level_values(0).unique()[0]:'ACM'}, level=0)
            heatmap['Month'] = sheet
            missing_cols = set(expected_cols) - set(heatmap)
#             print(f"{Path.WindowsPath(filepath).parts[-2]}: {sheet}: {missing_cols}")
            # Run this line to rename cols only if they match our expected names
#             heatmap = heatmap[[column for column in list(heatmap) if column in expected_cols]]
            # Remove columns containing "Color"
            heatmap = heatmap[[column for column in list(heatmap) if str(column).find("Color") == -1]]
            heatmap['Folder'] = Path.WindowsPath(filepath).parts[-2]
            heatmap = heatmap[0:13]
            # Run this line to rename cols assuming universal position
            heatmap.columns = declare_cols
            d.append(heatmap)
            
raw_heatmaps_df = pd.concat(d, ignore_index=True)

# heatmaps_df.to_csv(r"privatedata\OC_raw.csv", index=False)

In [ ]:
# Load from offline copy
# heatmaps_df = pd.read_csv(r"privatedata\OC_raw.csv")
heatmaps_df = raw_heatmaps_df
heatmaps_df = heatmaps_df.loc[~heatmaps_df['ACM'].isnull() &
                              (heatmaps_df['ACM']!='LLP Ratings') &
                              ~heatmaps_df['Plan Rating'].isnull() &
                              ~heatmaps_df['ET Rating'].isnull()]
heatmaps_df['Folder'] = heatmaps_df['Folder'].str.replace('Chicago ','')
heatmaps_df['Folder'] = heatmaps_df['Folder'].str.replace('Academy HS','Chicago Academy HS')

# Load schools from salesforce
account_df = get_cysh_df('Account', ['Id', 'Name'])
account_df.rename(columns={'Id':'Organization__c', 'Name':'School'}, inplace=True)

d = []
for folder_name in set(heatmaps_df['Folder']):
    match = process.extract(folder_name, set(account_df['School']), scorer=fuzz.token_set_ratio, limit=1)
    match = [folder_name] + [x for tup in match for x in tup]
    d.append(match)
df = pd.DataFrame(d, columns=['Folder', 'School', 'Match_Score'])
df.head()

# merge
heatmaps_df = heatmaps_df.merge(df[['Folder', 'School']], how='left', on='Folder')
heatmaps_df['Key'] = heatmaps_df['ACM'] + heatmaps_df['School']

# Load staff from salesforce, merge in school name
staff_df = get_cysh_df('Staff__c', ['Id', 'Name', 'Site__c', 'Organization__c', 'Reference_Id__c', 'Role__c'])
acm_df = staff_df[(staff_df['Site__c']=="Chicago") & staff_df['Role__c'].str.contains('Corps Member')]
acm_df = acm_df.merge(account_df, how='left', on='Organization__c')

# iterate by team, search for matches only within team
d=[]
for school in set(heatmaps_df['School']):
    raw_acm_names = set(heatmaps_df['ACM'].loc[heatmaps_df['School']==school])
    legit_acm_names = set(acm_df['Name'].loc[acm_df['School']==school])
    for name in raw_acm_names:
        match = process.extract(name, legit_acm_names, limit=2)
        match = [name] + [x for tup in match for x in tup] + [school]
        d.append(match)
matches_df = pd.DataFrame(d, columns=['ACM_raw', 'ACM', 'Match1_Score', 'ACM2', 'Match2_Score', 'School'])
matches_df['Key'] = matches_df['ACM_raw'] + matches_df['School']

del heatmaps_df['ACM']

heatmaps_df = heatmaps_df.merge(matches_df[['ACM', 'Key']], how='left', on='Key')

del heatmaps_df['Key'], heatmaps_df['Folder']

In [ ]:
heatmaps_df = heatmaps_df[['ACM',
                           'School',
                           'Month',
                           'Plan Rating',
                           'Plan Rubric Note',
                           'ET Rating',
                           'ET Rubric Note',
                           'ESE Rating',
                           'ESE Rubric Note',
                           'SPM Rating',
                           'SPM Rubric Note',
                           'Learn Rating',
                           'Learn Rubric Note']]

heatmaps_df.to_excel(r"Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\SY18 O&C Heat Map Rollup.xlsx",
                     index=False, freeze_panes=(1,0))

## Next Steps
* Clean month column
* Clean other sheets?
* Determine aggregation method